In [1]:
import numpy as np
import pandas as pd
import scanpy as sc
import anndata as ad
import scanpy.external as sce
import scirpy as ir
import decoupler as dc
# import scrublet as scr
import muon as mu
#from vpolo.alevin import parser # to parse alevin output
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm
import statsmodels.stats.multitest as multi
from adjustText import adjust_text
import re

seed = 0
np.random.seed(seed)
%matplotlib inline

import matplotlib as mpl
mpl.rcParams['figure.facecolor'] = (1,1,1,1)
mpl.rcParams['pdf.fonttype'] = 42
mpl.rcParams['ps.fonttype'] = 42

2024-12-20 09:42:43.027220: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-20 09:42:43.042670: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-20 09:42:43.057677: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-20 09:42:43.061962: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-20 09:42:43.076186: I tensorflow/core/platform/cpu_feature_guar

In [2]:
sc.settings.verbosity = 3  # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.settings.set_figure_params(dpi=80, color_map='viridis', transparent=False, frameon=False)  # low dpi (dots per inch) yields small inline figures

import matplotlib as mpl
# 2 lines below solved the facecolor problem.
# mpl.rcParams['figure.facecolor'] = 'white'
mpl.rcParams['figure.facecolor'] = (1,1,1,1)
sc.settings.autosave = True
sc.logging.print_header()

version = '241219_startrac'
# input_table = '../data/231009_PBMC_HH514.csv'

results_file = '../scanpy/{}/res.h5ad'.format(version)
results_file_mu = '../scanpy/{}/res.h5mu'.format(version)
results_file_cellxgene = '../scanpy/{}/res.cxg.h5ad'.format(version)

import os
os.makedirs('../scanpy/{}'.format(version), exist_ok=True)

sc.settings.figdir = '../scanpy/{}/graph'.format(version)
sc.settings.cachedir = '../scanpy/{}/cache'.format(version)
# %config InlineBackend.figure_format = 'retina'

import os
os.makedirs('../scanpy/{}'.format(version), exist_ok=True)
os.makedirs(sc.settings.figdir, exist_ok=True)

scanpy==1.10.2 anndata==0.10.9 umap==0.5.6 numpy==1.26.4 scipy==1.11.4 pandas==2.0.3 scikit-learn==1.5.1 statsmodels==0.14.2 igraph==0.11.6 pynndescent==0.5.13


In [3]:
adata_csf = sc.read('/home/yy693/pi_hafler/ASAP/scanpy/241028_CSF_downstream/res.h5ad')
adata_bld = sc.read('/home/yy693/pi_hafler/ASAP/scanpy/241028_BLD_downstream/res.h5ad')

In [4]:
adata = sc.concat([adata_bld, adata_csf])

In [5]:
adata.obs

,cellranger_sample_id,SampleID,SampleType,DonorID,Estimated Number of Cells,Mean Reads per Cell,Median Genes per Cell,Number of Reads,Valid Barcodes,Sequencing Saturation,...,majority_voting_Immune_All_Low,predicted_labels_COVID19_HumanChallenge_Blood,over_clustering_COVID19_HumanChallenge_Blood,majority_voting_COVID19_HumanChallenge_Blood,cluster_L1,cluster_L2,Hyposmia,Diagnosis_RBDstrat,Gender,Age
index,,,,,,,,,,,,,,,,,,,,,
AAACCTGAGCTCCTTC-1-YPD001BLD,YPD001BLD_HHT_GEX,YPD001BLD,BLD,YPD001,"6,931","47,589","2,450","329,839,529",90.1%,75.5%,...,Tem/Temra cytotoxic T cells,T CD8 EMRA,291,T CD8 EMRA,T,CD8CD4 Temra,False,HC,Male,79.0
AAACCTGAGGACATTA-1-YPD001BLD,YPD001BLD_HHT_GEX,YPD001BLD,BLD,YPD001,"6,931","47,589","2,450","329,839,529",90.1%,75.5%,...,Classical monocytes,Monocyte CD14+,284,Monocyte CD14+,Myeloid,CD14 Mono,False,HC,Male,79.0
AAACCTGAGGCCCTTG-1-YPD001BLD,YPD001BLD_HHT_GEX,YPD001BLD,BLD,YPD001,"6,931","47,589","2,450","329,839,529",90.1%,75.5%,...,CD16+ NK cells,NK,57,NK,T,CD8CD4 Temra,False,HC,Male,79.0
AAACCTGAGTCCCACG-1-YPD001BLD,YPD001BLD_HHT_GEX,YPD001BLD,BLD,YPD001,"6,931","47,589","2,450","329,839,529",90.1%,75.5%,...,Tem/Trm cytotoxic T cells,T CD8 EM,218,T CD8 EM,T,CD8 Tem,False,HC,Male,79.0
AAACCTGCACGAGAGT-1-YPD001BLD,YPD001BLD_HHT_GEX,YPD001BLD,BLD,YPD001,"6,931","47,589","2,450","329,839,529",90.1%,75.5%,...,Tem/Temra cytotoxic T cells,T CD4 CTL,104,T CD8 CTL,T,CD8CD4 Temra,False,HC,Male,79.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGTCATCGTTTATC-1-YPD028CSF,YPD028CSF_HHT_GEX,YPD028CSF,CSF,YPD028,"14,782","24,075","1,586","355,879,272",88.1%,71.3%,...,Tcm/Naive helper T cells,T CD4 Helper 0,446,T CD4 Helper 0,TNK,CD4 Tem,True,PD-RBD,Female,77.0
TTTGTCATCTCGCATC-1-YPD028CSF,YPD028CSF_HHT_GEX,YPD028CSF,CSF,YPD028,"14,782","24,075","1,586","355,879,272",88.1%,71.3%,...,Tcm/Naive helper T cells,T CD4 Helper 1,552,T CD4 Helper 1,TNK,CD4 Tem,True,PD-RBD,Female,77.0
TTTGTCATCTGATTCT-1-YPD028CSF,YPD028CSF_HHT_GEX,YPD028CSF,CSF,YPD028,"14,782","24,075","1,586","355,879,272",88.1%,71.3%,...,Tcm/Naive helper T cells,T CD4 Helper 2,94,T CD4 Helper 0,TNK,CD4 Tcm,True,PD-RBD,Female,77.0


In [6]:
del adata_csf
del adata_bld

In [7]:
f_file_list = '/home/yy693/pi_hafler/ASAP/data/241219_cellranger_files_VDJ.csv'


df_file = pd.read_csv(f_file_list)

list_cols = ['cellranger_sample_id', 'SampleID', 
       'SampleType', 'DonorID']
       
dict_adata_tcr = {}
for pos,row in df_file[df_file['vdj_type']=='TCR'].iterrows():
    a = ir.io.read_airr(f"{row['path']}/airr_rearrangement.tsv")
    a.var_names_make_unique()
    for col in list_cols:
        a.obs[col] = row[col]

    # sc.pp.scrublet(a, expected_doublet_rate=doublet_rate(a.shape[0], row['chemistry']))

    dict_adata_tcr[row['SampleID']] = a

adata_tcr = sc.concat(dict_adata_tcr, join='outer', index_unique='-')

dict_adata_bcr = {}
for pos,row in df_file[df_file['vdj_type']=='BCR'].iterrows():
    if int(row['Estimated Number of Cells'].replace(',','')) > 0:
        a = ir.io.read_airr(f"{row['path']}/airr_rearrangement.tsv")
        a.var_names_make_unique()
        for col in list_cols:
            a.obs[col] = row[col]

        # sc.pp.scrublet(a, expected_doublet_rate=doublet_rate(a.shape[0], row['chemistry']))

        dict_adata_bcr[row['SampleID']] = a

adata_bcr = sc.concat(dict_adata_bcr, join='outer', index_unique='-')

ir.pp.index_chains(adata_tcr)
ir.tl.chain_qc(adata_tcr)
adata_tcr = adata_tcr[adata_tcr.obs['receptor_type'] == 'TCR']

ir.pp.index_chains(adata_bcr)
ir.tl.chain_qc(adata_bcr)
adata_bcr = adata_bcr[adata_bcr.obs['receptor_type'] == 'BCR']

mdata = mu.MuData({"gex": adata, "tcr": adata_tcr, "bcr": adata_bcr})
mdata

/gpfs/gibbs/project/hafler/yy693/conda_envs/scanpy1.10.2/lib/python3.11/site-packages/anndata/utils.py:346: ExperimentalFeatureWarning: Support for Awkward Arrays is currently experimental. Behavior may change in the future. Please report any issues you may encounter!
  warnings.warn(msg, category, stacklevel=stacklevel)


/gpfs/gibbs/project/hafler/yy693/conda_envs/scanpy1.10.2/lib/python3.11/site-packages/anndata/utils.py:346: ExperimentalFeatureWarning: Outer joins on awkward.Arrays will have different return values in the future. For details, and to offer input, please see:

	https://github.com/scverse/anndata/issues/898
  warnings.warn(msg, category, stacklevel=stacklevel)


/gpfs/gibbs/project/hafler/yy693/conda_envs/scanpy1.10.2/lib/python3.11/site-packages/anndata/_core/aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


Filtering chains...
Indexing VJ chains...


/gpfs/gibbs/project/hafler/yy693/conda_envs/scanpy1.10.2/lib/python3.11/site-packages/anndata/_core/aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


Indexing VDJ chains...
build result array
Stored result in `adata.obs["receptor_type"]`.
Stored result in `adata.obs["receptor_subtype"]`.
Stored result in `adata.obs["chain_pairing"]`.
Filtering chains...
Indexing VJ chains...
Indexing VDJ chains...
build result array
Stored result in `adata.obs["receptor_type"]`.
Stored result in `adata.obs["receptor_subtype"]`.
Stored result in `adata.obs["chain_pairing"]`.


/gpfs/gibbs/project/hafler/yy693/conda_envs/scanpy1.10.2/lib/python3.11/site-packages/mudata/_core/mudata.py:1531: FutureWarning: From 0.4 .update() will not pull obs/var columns from individual modalities by default anymore. Set mudata.set_options(pull_on_update=False) to adopt the new behaviour, which will become the default. Use new pull_obs/pull_var and push_obs/push_var methods for more flexibility.
  self._update_attr("var", axis=0, join_common=join_common)
/gpfs/gibbs/project/hafler/yy693/conda_envs/scanpy1.10.2/lib/python3.11/site-packages/mudata/_core/mudata.py:1429: FutureWarning: From 0.4 .update() will not pull obs/var columns from individual modalities by default anymore. Set mudata.set_options(pull_on_update=False) to adopt the new behaviour, which will become the default. Use new pull_obs/pull_var and push_obs/push_var methods for more flexibility.
  self._update_attr("obs", axis=1, join_common=join_common)


MuData object with n_obs × n_vars = 1029222 × 23891
  3 modalities
    gex:	783081 x 23891
      obs:	'cellranger_sample_id', 'SampleID', 'SampleType', 'DonorID', 'Estimated Number of Cells', 'Mean Reads per Cell', 'Median Genes per Cell', 'Number of Reads', 'Valid Barcodes', 'Sequencing Saturation', 'Q30 Bases in Barcode', 'Q30 Bases in RNA Read', 'Q30 Bases in UMI', 'Reads Mapped to Genome', 'Reads Mapped Confidently to Genome', 'Reads Mapped Confidently to Intergenic Regions', 'Reads Mapped Confidently to Intronic Regions', 'Reads Mapped Confidently to Exonic Regions', 'Reads Mapped Confidently to Transcriptome', 'Reads Mapped Antisense to Gene', 'Fraction Reads in Cells', 'Total Genes Detected', 'Median UMI Counts per Cell', 'Q30 Bases in RNA Read 2', 'chemistry', 'Visit', 'SubjectID', 'DateICFSigned', 'YearICFSigned', 'EnrollmentCategory', 'RBDY/N?', 'YearofRBDDx', 'RBDAAO', 'AgeRBDDx', 'RBDDxtoConesnt', 'PDY/N', 'Year1stPDSx', 'YearofPDDx', 'PDAAO', 'AgePDDx', 'PDDxtoConsent', 'DLBY/N', 'Year1stDLBSx', 'YearDLBDX', 'DLBAAO', 'AgeDLBDx', 'DLBDxtoConsent', 'PSGinEpic?', 'BirthYear', 'MeetsMDSPDCriteria?', 'PSGDate', 'Ethnicity', 'Race', 'RaceOther', 'PDFamilyHx', 'PDFHMother', 'PDFHFather', 'PDFHFullSibs', 'PDFHHalfSibs', 'PDFHMaternalGPs', 'PDFHPaternalGPs', 'PDFHMaternalAuntsorUncles', 'PDFHChildren', 'PDFHOther', 'DLBFamilyHx', 'DLBFHMother', 'DLBFHFather', 'DLBFHFullSibs', 'DLBFHHalfSibs', 'DLBFHMaternalGPs', 'DLBFHPaternalGPs', 'DLBFHMaternalAuntsorUncles', 'DLBFHChildren', 'DLBFHOther', 'MSAFamilyHx', 'MSAFHMother', 'MSAFHFather', 'MSAFHFullSibs', 'MSAFHHalfSibs', 'MSAFHMaternalGPs', 'MSAFHPaternalGPs', 'MSAFHMaternalAuntsorUncles', 'MSAFHChildren', 'MSAFHOther', 'PDRiskPesticideExposure', 'PDRiskOccupationalsolventExposure', 'PDRiskDrinksCaffeinatedBeverages', 'PDRiskSmokerorFormerSmoker', 'PDRiskPhysicallyActive', 'MeanStriatumSBR', 'MeanStriatumSBR%ile', 'Caudate/PutamenRatio', 'AssymetryIndex', 'OrthostaticHypotension?', 'BloodyTap?', 'ClinicalCSFDataObtained?', 'CSFProtein', 'CSFGlucose', 'ClinicalCSFRBC', 'ClinicalCSFTotalNucleatedCells', 'ClinicalCSFGranulocytes', 'ClinicalCSFLymphocytes', 'CSFClinicalMonocytes', 'CSFVolume', 'CSFTotalCells', 'CSFCells/uL', 'CSFNotes', 'CSFSAA', 'UPSITRawScore', 'UPSITPercentile', 'MOCAVisuospatial', 'MOCA2Naming', 'MOCAMemoryTrial1', 'MOCAMemoryTrial2', 'MOCAAttention', 'MOCALanguage', 'MOCAAbstraction', 'MOCADelayedRecall', 'MOCAMemoryIndexScore', 'MOCAOrientation', 'MOCATotal', 'ParisArousalTotal', 'ConstipationCriteriaScore', 'BMSQ1RisingfromChair', 'BMSQ2Handwriting', 'BMSQ3VoiceSofter', 'BMSQ4PoorBalance', 'BMSQ5FeetGettingStuck', 'BMSQ6FaceLessExpressive', 'BMSQ7ArmsorLegsShaking', 'BMSQ8Buttons', 'BMSQ9Shuffling', 'BMSQ10MoveMoreSlowlythanOthersofAge', 'BMSQTotal', 'LRScore', 'ProbableProdromalPD?', 'SCOPATotal', 'SCPOPAManualEntry', 'NMSSTotal', 'MDS-UPDRS1.1cognition', 'MDS-UPDRS1.2Hallucinations', 'MDS-UPDRS1.3Mood', 'MDS-UPDRS1.4Anxiety', 'MDS-UPDRS1.5Apathy', 'MDS-UPDRS1.6DOPADysregulation', 'MDS-UPDRS1.7Sleep', 'MDS-UPDRS1.8DaytimeSleepiness', 'MDS-UPDRS1.9Pain', 'UMD-UPDRS1.10Urological', 'MDS-UPDRS1.11Constipation', 'MDS-UPDRS1.12OrthostaticDizziness', 'MDS-UPDRS1.13Fatigue', 'MDS-UPDRS2.1Speech', 'MDS-UPDRS2.2Drooling', 'MDS-UPDRS2.3Swallowing', 'MDS-UPDRS2.4Eating', 'MDS-UPDRS2.5Dressing', 'MDS-UPDRS2.6Hygeine', 'MDS-UPDRS2.7Handwriting', 'MDS-UPDRS2.8Hobbies', 'MDS-UPDRS2.9TurninginBed', 'MDS-UPDRS2.10Tremor', 'MDS-UPDRS2.11RisingfromChair,Car,Bed', 'MDS-UPDRS2.12Walking', 'MDS-UPDRS2.13FreezingofGait', 'MDS-UPDRSTakingMeds', 'MDS-UPDRS3.1Speech', 'MDS-UPDRS3.2Facies', 'MDS-UPDRS3.3NeckRigidity', 'MDS-UPDRS3.3RUERigidity', 'MDS-UPDRS3.3LUERidigity', 'MDS-UPDRS3.3RLERidigity', 'MDS-UPDRS3.3LLERigidty', 'MDS-UPDRS3.4FingerTapsR', 'MDS-UPDRS3.4FingerTapsL', 'MDS-UPDRS3.5GraspR', 'MDS-UPDRS3.5GraspL', 'MDS-UPDRS3.6RSMR', 'MDS-UPDRS3.6RSML', 'MDS-UPDRS3.7ToeTapR', 'MDS-UPDRS3.7ToeTapL', 'MDS-UPDRS3.8LegAgilityR', 'MDS-UPDRS3.8LegAgilityL', 'MDS-UPDRS3.9RisingFrom

In [8]:
mdata.obs

,gex:cellranger_sample_id,gex:SampleID,gex:SampleType,gex:DonorID,gex:Estimated Number of Cells,gex:Mean Reads per Cell,gex:Median Genes per Cell,gex:Number of Reads,gex:Valid Barcodes,gex:Sequencing Saturation,...,tcr:receptor_type,tcr:receptor_subtype,tcr:chain_pairing,bcr:cellranger_sample_id,bcr:SampleID,bcr:SampleType,bcr:DonorID,bcr:receptor_type,bcr:receptor_subtype,bcr:chain_pairing
AAACCTGAGCTCCTTC-1-YPD001BLD,YPD001BLD_HHT_GEX,YPD001BLD,BLD,YPD001,"6,931","47,589","2,450","329,839,529",90.1%,75.5%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AAACCTGAGGACATTA-1-YPD001BLD,YPD001BLD_HHT_GEX,YPD001BLD,BLD,YPD001,"6,931","47,589","2,450","329,839,529",90.1%,75.5%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AAACCTGAGGCCCTTG-1-YPD001BLD,YPD001BLD_HHT_GEX,YPD001BLD,BLD,YPD001,"6,931","47,589","2,450","329,839,529",90.1%,75.5%,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AAACCTGAGTCCCACG-1-YPD001BLD,YPD001BLD_HHT_GEX,YPD001BLD,BLD,YPD001,"6,931","47,589","2,450","329,839,529",90.1%,75.5%,...,TCR,TRA+TRB,extra VJ,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AAACCTGCACGAGAGT-1-YPD001BLD,YPD001BLD_HHT_GEX,YPD001BLD,BLD,YPD001,"6,931","47,589","2,450","329,839,529",90.1%,75.5%,...,TCR,TRA+TRB,single pair,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
GTCTTCGGTTCCACAA-1-YPD094BLD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,YPD094BLD_BCR_VHT_VDJ,YPD094BLD,BLD,YPD094,BCR,IGH+IGK,single pair
GTTTCTAAGCCCGAAA-1-YPD094BLD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,YPD094BLD_BCR_VHT_VDJ,YPD094BLD,BLD,YPD094,BCR,IGH+IGK,single pair
TCGAGGCTCTTGCATT-1-YPD094BLD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,YPD094BLD_BCR_VHT_VDJ,YPD094BLD,BLD,YPD094,BCR,IGH+IGL,single pair
TTCTCCTTCGTTACGA-1-YPD094BLD,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,YPD094BLD_BCR_VHT_VDJ,YPD094BLD,BLD,YPD094,BCR,IGH+IGK,single pair


In [9]:
# using default parameters, `ir_dist` will compute nucleotide sequence identity
ir.pp.ir_dist(mdata, airr_mod='tcr')
ir.tl.define_clonotypes(mdata, airr_mod='tcr', receptor_arms="all", dual_ir="primary_only")

Computing sequence x sequence distance matrix for VJ sequences.
Computing sequence x sequence distance matrix for VDJ sequences.
Initializing lookup tables. 


/gpfs/gibbs/project/hafler/yy693/conda_envs/scanpy1.10.2/lib/python3.11/site-packages/scirpy/ir_dist/__init__.py:252: ImplicitModificationWarning: Trying to modify attribute `._uns` of view, initializing view as actual.
  params.adata.uns[key_added] = result
/gpfs/gibbs/project/hafler/yy693/conda_envs/scanpy1.10.2/lib/python3.11/site-packages/anndata/_core/aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/gpfs/gibbs/project/hafler/yy693/conda_envs/scanpy1.10.2/lib/python3.11/site-packages/anndata/_core/aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


--> Done initializing lookup tables. (0:00:51)
Computing clonotype x clonotype distances.
--> Done computing clonotype x clonotype distances.  (0:00:02)
Stored result in `mdata.obs["tcr:clone_id"]`.
Stored result in `mdata.obs["tcr:clone_id_size"]`.


In [10]:
list(mdata.obs.columns)

['gex:cellranger_sample_id',
 'gex:SampleID',
 'gex:SampleType',
 'gex:DonorID',
 'gex:Estimated Number of Cells',
 'gex:Mean Reads per Cell',
 'gex:Median Genes per Cell',
 'gex:Number of Reads',
 'gex:Valid Barcodes',
 'gex:Sequencing Saturation',
 'gex:Q30 Bases in Barcode',
 'gex:Q30 Bases in RNA Read',
 'gex:Q30 Bases in UMI',
 'gex:Reads Mapped to Genome',
 'gex:Reads Mapped Confidently to Genome',
 'gex:Reads Mapped Confidently to Intergenic Regions',
 'gex:Reads Mapped Confidently to Intronic Regions',
 'gex:Reads Mapped Confidently to Exonic Regions',
 'gex:Reads Mapped Confidently to Transcriptome',
 'gex:Reads Mapped Antisense to Gene',
 'gex:Fraction Reads in Cells',
 'gex:Total Genes Detected',
 'gex:Median UMI Counts per Cell',
 'gex:Q30 Bases in RNA Read 2',
 'gex:chemistry',
 'gex:Visit',
 'gex:SubjectID',
 'gex:DateICFSigned',
 'gex:YearICFSigned',
 'gex:EnrollmentCategory',
 'gex:RBDY/N?',
 'gex:YearofRBDDx',
 'gex:RBDAAO',
 'gex:AgeRBDDx',
 'gex:RBDDxtoConesnt',
 'ge

In [11]:
mdata.obs[['gex:EnrollmentCategory', 'gex:Diagnosis_RBDstrat']]

,gex:EnrollmentCategory,gex:Diagnosis_RBDstrat
AAACCTGAGCTCCTTC-1-YPD001BLD,HC,HC
AAACCTGAGGACATTA-1-YPD001BLD,HC,HC
AAACCTGAGGCCCTTG-1-YPD001BLD,HC,HC
AAACCTGAGTCCCACG-1-YPD001BLD,HC,HC
AAACCTGCACGAGAGT-1-YPD001BLD,HC,HC
...,...,...
GTCTTCGGTTCCACAA-1-YPD094BLD,NaN,NaN
GTTTCTAAGCCCGAAA-1-YPD094BLD,NaN,NaN
TCGAGGCTCTTGCATT-1-YPD094BLD,NaN,NaN
TTCTCCTTCGTTACGA-1-YPD094BLD,NaN,NaN


In [12]:
list_t_crosstissue = ['Tem/Temra cytotoxic T cells',
 'Tem/Trm cytotoxic T cells',
 'Regulatory T cells',
 'Tcm/Naive helper T cells',
 'Tem/Effector helper T cells',
 'Tcm/Naive cytotoxic T cells',]

df_startrac = mdata.obs[['tcr:clone_id', 'gex:DonorID', 'gex:majority_voting_Immune_All_Low', 'gex:SampleType',
'gex:EnrollmentCategory', 'gex:Diagnosis_RBDstrat']].dropna().reset_index()
df_startrac.columns = ['Cell_Name', 'clone.id', 'patient', 'majorCluster', 'loc', 'disease', 'disease_rbd']
# df_startrac['loc'] = 'CSF'
df_startrac = df_startrac[df_startrac['majorCluster'].isin(list_t_crosstissue)]
df_startrac.to_csv(f'../scanpy/{version}/startrac_input_T_crosstissue.csv')
df_startrac

,Cell_Name,clone.id,patient,majorCluster,loc,disease,disease_rbd
0,AAACCTGAGTCCCACG-1-YPD001BLD,0,YPD001,Tem/Trm cytotoxic T cells,BLD,HC,HC
1,AAACCTGCACGAGAGT-1-YPD001BLD,1,YPD001,Tem/Temra cytotoxic T cells,BLD,HC,HC
2,AAACCTGGTGAAGGCT-1-YPD001BLD,2,YPD001,Tem/Trm cytotoxic T cells,BLD,HC,HC
3,AAACGGGAGATCACGG-1-YPD001BLD,3,YPD001,Regulatory T cells,BLD,HC,HC
4,AAACGGGCAAGAGGCT-1-YPD001BLD,4,YPD001,Tem/Temra cytotoxic T cells,BLD,HC,HC
...,...,...,...,...,...,...,...
418057,TTTGTCATCGTTTATC-1-YPD028CSF,204471,YPD028,Tcm/Naive helper T cells,CSF,PD-RBD,PD-RBD
418058,TTTGTCATCTCGCATC-1-YPD028CSF,202374,YPD028,Tcm/Naive helper T cells,CSF,PD-RBD,PD-RBD
418059,TTTGTCATCTGATTCT-1-YPD028CSF,196485,YPD028,Tcm/Naive helper T cells,CSF,PD-RBD,PD-RBD
418060,TTTGTCATCTGGTGTA-1-YPD028CSF,204472,YPD028,Tcm/Naive helper T cells,CSF,PD-RBD,PD-RBD
